In [1]:
import pandas as pd
import h5py
import os
import h5py
import os

from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew, kurtosis
from tensorflow.keras.utils import to_categorical

def zero_crossing_rate(arr):
    """
    Sinyalde işaret değişimlerinin sayısını normalize ederek döndürür.
    """
    return np.sum(np.diff(np.sign(arr)) != 0) / len(arr)

In [2]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import cv2
import numpy as np

# Mobilenet modeli sadece feature çıkarımı için
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
feature_model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

/var/folders/kw/3t8qd9fd6hlfbqdps6pvpchw0000gn/T/ipykernel_60414/841802844.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))


In [3]:

X = []
y = []

for file in os.listdir("gestureData/"):
    if file.endswith(".mat"):
        user_id = int(file.split("_")[2])
        with h5py.File(f"gestureData/{file}", "r") as f:
            key = list(f.keys())[0]  # genellikle 'depth_sequence'
            sequence = f[key][:]  # [frames, height, width]
            
            # Sadece ilk 10 kare, her kareyi 64x64'e yeniden boyutlandır
            if sequence.shape[0] < 10:
                continue
            
            # frames = sequence[:10]
            # resized = [cv2.resize(frame, (64, 64)) for frame in frames]
            # stacked = np.stack(resized, axis=0)  # [25, 64, 64]

            frame_features = []
            for frame in sequence:
                resized = cv2.resize(frame, (64, 64))
                if len(resized.shape) == 2:
                    resized = np.stack([resized]*3, axis=-1)  # grayscale → RGB
                resized = resized.astype(np.float32) / 255.0
                feat = feature_model.predict(resized[np.newaxis, ...], verbose=0)
                frame_features.append(feat[0])
            
            frame_features = np.array(frame_features)  # [num_frames, 1280]
            mean = np.mean(frame_features, axis=0)
            std = np.std(frame_features, axis=0)
            median = np.median(frame_features, axis=0)
            # skewnes = skew(frame_features)
            # zcr = zero_crossing_rate(frame_features)
            concatenated = np.concatenate([mean, std, median])  # [2560]

            X.append(concatenated)
            y.append(user_id)


In [6]:
X = np.array(X)  # shape: 
y = np.array(y)

# CNN için channel ekle (örneğin grayscale için kanal sayısı = 1)
X = X[..., np.newaxis]  # 
X = X.reshape(333, 3840)

In [5]:
X.shape

(333, 3840, 1)

In [ ]:
len(y)

333

In [7]:
# y zaten elinde numpy array olarak var
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(np.unique(y_encoded))  # 10 sınıf
y_cat = to_categorical(y_encoded, num_classes=num_classes)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Eğitim/test böl
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(3840,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, epochs=35, batch_size=16, validation_data=(X_test, y_test))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 512)            │     1,966,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,100,490 (8.01 MB)

 Trainable params: 2,100,490 (8.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.1580 - loss: 2.7914 - val_accuracy: 0.1045 - val_loss: 2.4508
Epoch 2/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2134 - loss: 2.3676 - val_accuracy: 0.1343 - val_loss: 2.3081
Epoch 3/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2234 - loss: 2.2696 - val_accuracy: 0.1493 - val_loss: 2.2751
Epoch 4/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2586 - loss: 2.0988 - val_accuracy: 0.2537 - val_loss: 2.1405
Epoch 5/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3537 - loss: 1.8690 - val_accuracy: 0.2239 - val_loss: 2.2165
Epoch 6/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3876 - loss: 1.6869 - val_accuracy: 0.2687 - val_loss: 2.1930
Epoch 7/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4605 - loss: 1.7625 - val_accuracy: 0.2239 - val_loss: 2.1133
Epoch 8/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4742 - loss: 1.6932 - val_accuracy: 0.2090 - val_loss

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_simple, y_test_simple = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train_simple)

y_pred = clf.predict(X_test)
print(classification_report(y_test_simple, y_pred))

# Accuracy
acc = accuracy_score(y_test_simple, y_pred)
print(f"✅ Accuracy: {acc:.4f}")

              precision    recall  f1-score   support

           1       0.25      0.50      0.33         4
           2       0.67      0.40      0.50         5
           3       0.00      0.00      0.00         5
           4       0.50      0.67      0.57         6
           5       0.56      0.62      0.59         8
           6       0.20      0.33      0.25         3
           7       0.50      0.55      0.52        11
           8       0.67      0.22      0.33         9
           9       0.86      0.86      0.86         7
          10       0.75      0.67      0.71         9

    accuracy                           0.51        67
   macro avg       0.49      0.48      0.47        67
weighted avg       0.55      0.51      0.50        67

✅ Accuracy: 0.5075
